This notebook contains code related to some basic data preprocessing and data cleaning. It analyses the existing dataframe and then make chnages and save in a new file.

In [17]:
import pandas as pd

def load_and_print_data(file_path):

    # Load the dataset
    df = pd.read_csv(file_path)

    # Print the dataframe before pre-processing
    print("Dataframe before pre-processing:")
    print(df.head())

    return df


In [18]:
# Example usage
file_path = "/content/insurance_claims.csv"
df_old = load_and_print_data(file_path)

Dataframe before pre-processing:
   months_as_customer  age  policy_number policy_bind_date policy_state  \
0                 328   48         521585       2014-10-17           OH   
1                 228   42         342868       2006-06-27           IN   
2                 134   29         687698       2000-09-06           OH   
3                 256   41         227811       1990-05-25           IL   
4                 228   44         367455       2014-06-06           IL   

  policy_csl  policy_deductable  policy_annual_premium  umbrella_limit  \
0    250/500               1000                1406.91               0   
1    250/500               2000                1197.22         5000000   
2    100/300               2000                1413.14         5000000   
3    250/500               2000                1415.74         6000000   
4   500/1000               1000                1583.91         6000000   

   insured_zip  ... witnesses police_report_available total_claim_amoun

In [19]:
def preprocess_insurance_data(df):

    # Replace '?' with NaN for proper handling of missing values
    df.replace('?', pd.NA, inplace=True)

    # Convert categorical target variable to binary (Y -> 1, N -> 0)
    df['fraud_reported'] = df['fraud_reported'].map({'Y': 1, 'N': 0})

    # Convert date columns to datetime format
    df['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'], errors='coerce')
    df['incident_date'] = pd.to_datetime(df['incident_date'], errors='coerce')

    # Drop unnecessary columns
    drop_columns = ['policy_number', 'insured_zip', 'incident_location', 'auto_make', 'auto_model']
    df.drop(columns=drop_columns, inplace=True, errors='ignore')

    # Handle missing values by replacing categorical NaNs with 'Unknown'
    categorical_cols = df.select_dtypes(include=['object']).columns
    df[categorical_cols] = df[categorical_cols].fillna('Unknown')

    # Encode categorical variables using one-hot encoding
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

    return df


In [20]:
# Example usage
df_cleaned = preprocess_insurance_data(df_old)

# Print the dataframe after pre processing
print("\nDataframe after pre-processing:")
print(df_cleaned.head())

# Save the preprocessed dataset
df_cleaned.to_csv("insurance_claims_preprocessed.csv", index=False)


Dataframe after pre-processing:
   months_as_customer  age policy_bind_date  policy_deductable  \
0                 328   48       2014-10-17               1000   
1                 228   42       2006-06-27               2000   
2                 134   29       2000-09-06               2000   
3                 256   41       1990-05-25               2000   
4                 228   44       2014-06-06               1000   

   policy_annual_premium  umbrella_limit  capital-gains  capital-loss  \
0                1406.91               0          53300             0   
1                1197.22         5000000              0             0   
2                1413.14         5000000          35100             0   
3                1415.74         6000000          48900        -62400   
4                1583.91         6000000          66000        -46000   

  incident_date  incident_hour_of_the_day  ...  incident_city_Columbus  \
0    2015-01-25                         5  ...           

The changes made to the dataset,
1. **Handling Missing Values:**  
   - Replaced `?` with `NaN` for proper identification of missing data.  
   - Filled missing categorical values with `'Unknown'`.  

2. **Target Variable Conversion:**  
   - Converted `fraud_reported` from categorical (`Y/N`) to binary (`1/0`).  

3. **Date Format Conversion:**  
   - Converted `policy_bind_date` and `incident_date` into proper `datetime` format.  

4. **Dropped Unnecessary Columns:**  
   - Removed `policy_number`, `insured_zip`, `incident_location`, `auto_make`, and `auto_model` as they were unlikely to contribute to fraud detection.  

5. **Encoding Categorical Variables:**  
   - Applied one-hot encoding to categorical columns while avoiding dummy variable traps by dropping the first category.  

6. **Final Dataset Preparation:**  
   - Processed dataset was saved as `insurance_claims_preprocessed.csv` for further model training.  


In [21]:
# Handling outliers
import numpy as np

# Function to handle outliers using IQR method
def handle_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Removing outliers
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# Function to handle outliers by capping (Winsorization)
def handle_outliers_capping(df, col):
    lower_bound = df[col].quantile(0.05)
    upper_bound = df[col].quantile(0.95)

    # Capping the outliers
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    return df

# Function to handle outliers by log transformation
def handle_outliers_log(df, col):
    df[col] = np.log1p(df[col])  # log1p to avoid log(0)
    return df

# Applying IQR method to extreme value columns
df_cleaned = handle_outliers_iqr(df_cleaned, 'policy_annual_premium')
df_cleaned = handle_outliers_iqr(df_cleaned, 'total_claim_amount')

df_cleaned = handle_outliers_capping(df_cleaned, 'injury_claim')
df_cleaned = handle_outliers_capping(df_cleaned, 'property_claim')
df_cleaned = handle_outliers_capping(df_cleaned, 'vehicle_claim')

df_cleaned = handle_outliers_log(df_cleaned, 'capital-gains')
df_cleaned = handle_outliers_log(df_cleaned, 'capital-loss')

# Displaying the final dataframe after outlier handling
print(df_cleaned.describe())

# Saving the updated dataframe to a new CSV file
df_cleaned.to_csv('insurance_claims_outliers_handled.csv', index=False)


       months_as_customer         age               policy_bind_date  \
count          990.000000  990.000000                            990   
mean           204.355556   38.954545  2002-02-13 10:03:38.181818240   
min              0.000000   19.000000            1990-01-08 00:00:00   
25%            117.250000   32.000000            1995-09-23 18:00:00   
50%            200.000000   38.000000            2002-04-01 12:00:00   
75%            276.750000   44.000000            2008-04-22 12:00:00   
max            479.000000   64.000000            2015-02-22 00:00:00   
std            114.638399    9.120969                            NaN   

       policy_deductable  policy_annual_premium  umbrella_limit  \
count         990.000000             990.000000    9.900000e+02   
mean         1137.878788            1256.002960    1.103030e+06   
min           500.000000             617.110000   -1.000000e+06   
25%           500.000000            1090.402500    0.000000e+00   
50%          100

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


On analysing the dataset we realised that the columns such as `months_as_customer` , `age` , `policy_annual_premium` ,  `capital-gains / capital-loss` , `total_claim_amount` , `injury_claim` , `property_claim` , `vehicle_claim` could behve as outliers.
So, we used  different approaches for different types of columns that are as follows :

    IQR Method: For columns where outliers seem like data errors (policy premium).

    Capping (95th Percentile): For injury, property, and vehicle claims where extreme values are valid but need limiting.

    Log Transformation: For capital gains/loss to normalize extreme skewness.